In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
csv_path = "Resources/videos.csv"
df = pd.read_csv(csv_path)
df.head(10)

In [ ]:
# Mike's code
test_list = [0, 1, 2, 3]

In [ ]:
# Riley's code

In [ ]:
# Testing testing 123
apple = 1
or

In [ ]:
# Jackie's code
df['trending_date']=pd.to_datetime(df['trending_date'].astype(str), format='%Y.%m.%d')
df['trending_date'] = df['trending_date'].dt.strftime('%m/%d/%Y')
df.head()

In [ ]:
# Brian's code

In [ ]:
#this is my test cell to merge